<a href="https://colab.research.google.com/github/david-meltzer/LLMs/blob/main/llama_baurm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dependecies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/SFT

/content/drive/MyDrive/SFT


In [3]:
%ls

artifacts/  finetuned_models/  llama.ipynb  requirements.txt  wandb/


In [4]:
!pip install -qqqU -r requirements.txt
!pip install -qqqU datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch, datasets, accelerate, transformers, evaluate, gc
import bitsandbytes as bnb

from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          GenerationConfig,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          BitsAndBytesConfig)

from peft import (LoraConfig,
                  get_peft_model,
                  prepare_model_for_kbit_training)

from trl import (SFTTrainer,
                 DataCollatorForCompletionOnlyLM)

from google.colab import runtime


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [6]:
from pprint import pprint

def pp(x):
  pprint(x, compact = True, width = 150)

def cleanup(x):
    if x == 'memory':
        gc.collect()
        torch.cuda.empty_cache()
    elif x == 'runtime':
        runtime.unassign()

In [7]:
from huggingface_hub import login
login()

In [8]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Dataset

In [ ]:
wandb_api = wandb.Api()
artifact = wandb_api.artifact('ft-llmmm/ELI5_analysis/combined_dataset:v5', type='dataset')
artifact_dir = artifact.download()

In [9]:
artifact_dir = './artifacts/combined_dataset:v5'

In [10]:
from datasets import load_from_disk

data = load_from_disk(artifact_dir)
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA'],
        num_rows: 72214
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA'],
        num_rows: 1964
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA'],
        num_rows: 3301
    })
})

In [11]:
#Add QA number of words to the dataset

def add_num_words(example):
    example['num_words'] = len(example['QA'].split())
    return example

In [12]:
data = data.map(add_num_words)
data = data.filter(lambda x : x['num_words'] < 400)
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 68587
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 1890
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 3107
    })
})

In [13]:
wiki = data.filter(lambda example : example['source'] == 'simple_wiki')
wiki

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 29987
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 1000
    })
})

In [14]:
eli5 = data.filter(lambda example : example['source'] == 'ELI5')
eli5

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 38600
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 890
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 2107
    })
})

#Tokenizer, formatting, collator

In [15]:
model_id = 'meta-llama/Llama-2-7b-hf'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [16]:
# set up the collator

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Human: {example['question'][i]}\n ### Assistant: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

response_template = "### Assistant:"
collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)

In [ ]:
# data_tokenized = data.map(lambda x : tokenizer(x['QA'], return_length = True))
# max_seq_length = max(data_tokenized['train']['length'])[0]

# wiki_tokenized = wiki.map(lambda x : tokenizer(x['QA'], return_length = True))
# max_seq_length_wiki = max(wiki_tokenized['train']['length'])[0]

# eli5_tokenized = wiki.map(lambda x : tokenizer(x['QA'], return_length = True))
# max_seq_length_eli5 = max(eli5_tokenized['train']['length'])[0]

##Word length statistics

In [ ]:
# data_sorted = data.sort('num_words')

# import numpy as np
# len_stat = np.zeros((3,400))
# split_names = {'train' : 0, 'validation' : 1 , 'test' : 2}

# for k, v in split_names.items():
#     lengths = np.array(data_sorted[k]['num_words'])
#     len_stat[v] = np.array([lengths[lengths ==n].shape[0] for n in range(400)] )

# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1,3, figsize=(10, 3))
# fig.suptitle('Number of words distibution')

# for k, v  in split_names.items():

#   axs[v].plot(len_stat[v, :])
#   axs[v].set_title(k + ' dataset')
#   if v ==0:
#     axs[v].set_xlabel('Number of words')
#     axs[v].set_ylabel('Number of examples')

# plt.tight_layout()

#Inference analysis

In [ ]:
# from peft import PeftModel

# model_base = AutoModelForCausalLM.from_pretrained(model_id)

# model_trained = PeftModel.from_pretrained(model = model_base,
#                     model_id = 'finetuned_models/checkpoint-1170/',
#                     torch_dtype = torch.bfloat16,
#                     is_trainable = False)

# model_trained_merged = model_trained.merge_and_unload()

In [ ]:
# import random
# random.seed(123)

# def generate_examples(models, n):
#     """
#     Inputs:
#         models -- list, a list of models
#         n      -- int, number of examples to generate
#     """

#     indices = [random.randint(0, len(wiki_qa['test']['question']) ) for _ in range(n)]



#     dash_line = '-'*150

#     #for model in models:
#     #    model.config.pad_token_id = model.config.eos_token_id

#     generation_config = GenerationConfig(   num_beams = 3,
#                                             #max_length = 100,
#                                             max_new_tokens = 50,
#                                             do_sample = True,
#                                             temperature = 0.5,
#                                             top_k = 50,
#                                             top_p = 0.8,
#                                             repetition_penalty = 1.0,
#                                             #pad_token_id = model.config.eos_token_id
#                                         )


#     for idx in indices:
#         question = wiki_qa['test'][idx]['question']
#         answer = wiki_qa['test'][idx]['trunc_text']

#         prompt = f"### Question: {question}\n### Answer:"
#         input = tokenizer(prompt, return_tensors = 'pt', padding = False)


#         print(dash_line)
#         print(f'Input prompt:\n{prompt}')
#         print(dash_line)
#         print(f'Baseline:\n{answer}')
#         print(dash_line)

#         for i in range(len(models)):
#             model = models[i]
#             output_ids = model.generate(input_ids = input['input_ids'],
#                                         attention_mask = input['attention_mask'],
#                                         generation_config = generation_config,
#                                         pad_token_id = model.config.eos_token_id,
#                                         #stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=100)])
#                                         )

#             output = tokenizer.decode(output_ids[0], skip_special_tokens = True)

#             print(f'Model {i+1} output:\n{output}')
#             print(dash_line)

#         print('\n\n')

In [ ]:
# generate_examples([model_base, model_trained, model_trained_merged],3)

In [ ]:
# idx = 300
# model = model_base
# question = wiki_qa['test'][idx]['question']
# prompt = f"### Question: {question}\n### Answer:"
# input = tokenizer(prompt, return_tensors = 'pt', padding = False)

# output_ids = model.generate(input_ids = input['input_ids'],
#                             #attention_mask = input['attention_mask'],
#                             generation_config = GenerationConfig(   num_beams = 5,
#                                                                     max_length = 100,
#                                                                     max_new_tokens = 50,
#                                                                     do_sample = True,
#                                                                     temperature = 2.5,
#                                                                     top_k = 40,
#                                                                     top_p = 0.5,
#                                                                     #repetition_penalty = 1.0,
#                                                                     #pad_token_id = model.config.eos_token_id
#                                                                 ),
#                             pad_token_id = model.config.eos_token_id,
#                             eos_token_id = model.config.eos_token_id
#                             #stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=100)])
#                             )
# output = tokenizer.decode(output_ids[0], skip_special_tokens = True)
# print(output)

#Model setup

In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [18]:
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

In [19]:
def prepare_quant_model(model_id = model_id):

    nf4_config = BitsAndBytesConfig(load_in_4bit=True,
                                    bnb_4bit_quant_type="nf4",
                                    bnb_4bit_use_double_quant=True,
                                    bnb_4bit_compute_dtype=torch.bfloat16
                                    )

    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                quantization_config=nf4_config,
                                                device_map='auto',
                                                torch_dtype = torch.bfloat16
                                                )

    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    return model

def get_lora_model(model, lora_rank = 32):

    config = LoraConfig(r=lora_rank,
                        lora_alpha=32,
                        target_modules=find_all_linear_names(model),
                        lora_dropout=0.1,
                        bias="none",
                        task_type="CAUSAL_LM")

    model = get_peft_model(model, config)

    return model

#Training

In [20]:
%env WANDB_PROJECT=sft_llama_bm
WANDB_LOG_MODEL="checkpoint"

env: WANDB_PROJECT=sft_llama_bm


In [ ]:
lr_schedule = timm.scheduler.CosineLRScheduler()

In [21]:
def training_args_config(lora_rank = 32,
                         num_epochs = 1,
                         lr = 1e-3,
                         weight_decay = 0.1,
                         batch_size = 8,
                         gradient_accumulation_steps = 4,
                         eval_steps = 500,
                         find_batch_size = True,
                         lr_schedule = 'linear',
                         output_dir = 'finetuned_models/llama',
                         run_name = None
                         ):

    params = {

    #-----training----------------
    'num_train_epochs' : num_epochs,
    'learning_rate' : lr,
    'weight_decay' : weight_decay,
    'dataloader_num_workers' : 4,
    'per_device_train_batch_size' : batch_size,
    'per_device_eval_batch_size' : batch_size,
    'group_by_length' : True,
    'auto_find_batch_size' : find_batch_size,
    'lr_scheduler_type' : lr_schedule,

    #-----efficient training-----
    'gradient_accumulation_steps' : gradient_accumulation_steps,
    'gradient_checkpointing' : True,
    'optim' : 'paged_adamw_8bit',
    'bf16' : True,

    #-----logging----------------
    'report_to' : 'wandb',
    'output_dir' : output_dir,

    'evaluation_strategy' : "steps",
    'eval_steps' : eval_steps,

    'logging_strategy' : 'steps',
    'logging_steps' : eval_steps,

    'save_strategy' : "steps",
    'save_steps' : eval_steps,

    'save_total_limit' : 2,
    'load_best_model_at_end' : True,
    'metric_for_best_model' : "eval_loss",
    'greater_is_better' : False,

    'run_name' : f'r{lora_rank}_lr{lr:.0e}_wd{weight_decay}_bs{batch_size}' + ('_' + run_name if run_name is not None else ''),
    }

    params['output_dir'] = params['output_dir'] + '/' + params['run_name']

    return params

In [22]:
#model_id = 'meta-llama/Llama-2-7b-hf'
r = 16

model= prepare_quant_model(model_id = model_id)
model = get_lora_model(model, lora_rank = r)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
tot_steps = wiki['train'].num_rows / (26 * 4)
tot_steps/3

96.11217948717949

In [23]:
hyperparams = training_args_config( lora_rank = r,
                                    num_epochs = 3,
                                    lr = 1e-5,
                                    weight_decay = 0.1,
                                    batch_size = 26,
                                    gradient_accumulation_steps = 4,
                                    eval_steps = 96,
                                    find_batch_size = True,
                                    lr_schedule = 'linear',
                                    output_dir = 'finetuned_models/llama',
                                    run_name = 'wiki'
                                    )
pp(hyperparams)

trainer = SFTTrainer(   model,
                        args = TrainingArguments(**hyperparams),
                        max_seq_length=1024,
                        train_dataset=wiki['train'],
                        eval_dataset=wiki['validation'],
                        formatting_func=formatting_prompts_func,
                        data_collator=collator,
)

Using pad_token, but it is not set yet.


{'auto_find_batch_size': True,
 'bf16': True,
 'dataloader_num_workers': 4,
 'eval_steps': 96,
 'evaluation_strategy': 'steps',
 'gradient_accumulation_steps': 4,
 'gradient_checkpointing': True,
 'greater_is_better': False,
 'group_by_length': True,
 'learning_rate': 1e-05,
 'load_best_model_at_end': True,
 'logging_steps': 96,
 'logging_strategy': 'steps',
 'lr_scheduler_type': 'linear',
 'metric_for_best_model': 'eval_loss',
 'num_train_epochs': 3,
 'optim': 'paged_adamw_8bit',
 'output_dir': 'finetuned_models/llama/r16_lr1e-05_wd10_bs26_wiki',
 'per_device_eval_batch_size': 26,
 'per_device_train_batch_size': 26,
 'report_to': 'wandb',
 'run_name': 'r16_lr1e-05_wd10_bs26_wiki',
 'save_steps': 96,
 'save_strategy': 'steps',
 'save_total_limit': 2,
 'weight_decay': 10}


In [24]:
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


In [25]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
96,1.303600,1.283455
192,1.269600,1.267538
288,1.255100,1.260830
384,1.220300,1.259081
480,1.216600,1.255147
576,1.232400,1.253440
672,1.183800,1.256996
768,1.183100,1.255568
864,1.188500,1.255063


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using t

TrainOutput(global_step=864, training_loss=1.2281091919651739, metrics={'train_runtime': 7317.2121, 'train_samples_per_second': 12.294, 'train_steps_per_second': 0.118, 'total_flos': 2.787798350594949e+17, 'train_loss': 1.2281091919651739, 'epoch': 2.99})

In [26]:
wandb.finish()

eval/loss,█▄▃▂▁▁▂▁▁
eval/runtime,▁█▇▅▂█▇▇▄
eval/samples_per_second,█▁▂▄▇▁▂▂▅
eval/steps_per_second,█▁▃▆█▁▁▁▆
train/epoch,▁▁▂▂▃▃▄▄▄▄▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▅▄▃▂▁
train/loss,█▆▅▃▃▄▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [29]:
del model
cleanup('memory')

In [27]:
cleanup('runtime')